Импортируем библиотеки

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import catboost as cb
from datetime import datetime
import seaborn as sns

In [2]:
data_train = pd.read_csv("./data/train.csv")
data_test = pd.read_csv("./data/test.csv")
data_webstat = pd.read_csv("./data/t1_webstat.csv")

is_callcenter - был ли совершен звонок в коллцентр по итогам сессии (нужно предсказать)

order_id - id заказа

create_time - время создания заказа

good_id - id товара

price - цена товара

utm_medium - тип трафика

utm_source - источник трафика

sessionkey_id - id сессии пользователя

category_id - id категории товара

parent_id - id родительской категории для категории товара

root_id - id "базовой" категории товара

rating_value - рейтинг товара

rating_count - число оценок товара

description_length - длина описания товара

goods_qty - число товаров в заказе

model_id - ???

is_moderated - ???

pics_qty - ???

model_create_time - ???

sessionkey_id - id сессии пользователя

date_time - timestamp захода на страницу определенного типа

page_type - тип просмотренной страницы

pageview_number - порядковый номер страницы среди всех страниц, отсортированных по времени посещения внутри сессии

pageview_duration_sec - время просмотра страницы

product_in_sale - ???

In [3]:
data_train["create_time_ts"] = None
data_test["create_time_ts"] = None
data_train["model_create_time_ts"] = None
data_test["model_create_time_ts"] = None

from datetime import datetime
# convert string to date
data_train.create_time_ts = data_train.apply(lambda x: datetime.strptime(x.create_time[:19], '%Y-%m-%d %H:%M:%S'), axis=1)
data_test.create_time_ts = data_test.apply(lambda x: datetime.strptime(x.create_time[:19], '%Y-%m-%d %H:%M:%S'), axis=1)

data_train.loc[~data_train.model_create_time.isna(), "model_create_time_ts"] = data_train.loc[~data_train.model_create_time.isna()].apply(lambda x: datetime.strptime(x.model_create_time[:19], '%Y-%m-%d %H:%M:%S'), axis=1)
data_test.loc[~data_test.model_create_time.isna(), "model_create_time_ts"] = data_test.loc[~data_test.model_create_time.isna()].apply(lambda x: datetime.strptime(x.model_create_time[:19], '%Y-%m-%d %H:%M:%S'), axis=1)

In [4]:
del data_train["create_time"]
del data_train["model_create_time"]
del data_test["create_time"]
del data_test["model_create_time"]

In [5]:
data_train.model_create_time_ts = data_train.model_create_time_ts.fillna(data_train.create_time_ts)
data_test.model_create_time_ts = data_test.model_create_time_ts.fillna(data_test.create_time_ts)

In [123]:
print("Рейтинг товаров, по которым позвонили: ", data_train[data_train["is_callcenter"] == 1]["rating_value"].mean(), 
      ", по которым не звонили", data_train[data_train["is_callcenter"] == 0]["rating_value"].mean())

Рейтинг товаров, по которым позвонили:  1.5399876007439555 , по которым не звонили 1.4771690174232548


In [124]:
print("Длина описания товаров, по которым позвонили: ", data_train[data_train["is_callcenter"] == 1]["description_length"].mean(), 
      ", по которым не звонили", data_train[data_train["is_callcenter"] == 0]["description_length"].mean())

Длина описания товаров, по которым позвонили:  572.92983638373 , по которым не звонили 573.338671921299


In [125]:
print("Представителей 0 класса: 67496, 1 класса: 37099")

Представителей 0 класса: 67496, 1 класса: 37099


In [15]:
A = data_train.groupby(['root_id'])['is_callcenter'].sum().sort_values()
B = data_train.groupby(['root_id'])['is_callcenter'].count().sort_values()
C = A.divide(B).sort_values()

order_id = list(data_train["order_id"].unique())
new_column = list()
for order in order_id:
    root = list(data_train[data_train["order_id"] == order]["root_id"].values)[0]
    new_column.append(C[root])
new_column_test = list()
for order in list(data_test["order_id"].unique()):
    root = list(data_test[data_test["order_id"] == order]["root_id"].values)[0]
    new_column_test.append(C[root])
data_train['root_mean_call'] = new_column
data_test['root_mean_call'] = new_column_test

In [666]:
new_column = list()
for order in range(data_train.shape[0]):
    hour = data_train["create_time_ts"][order].time().hour * 60 + data_train["create_time_ts"][order].time().minute
    new_column.append(hour)
data_train['hour'] = new_column
new_column = list()
for order in range(data_test.shape[0]):
    hour = data_train["create_time_ts"][order].time().hour * 60 + data_train["create_time_ts"][order].time().minute
    new_column.append(hour)
data_test['hour'] = new_column

In [54]:
data_train["utm_source"].fillna(0, inplace=True)
data_test["utm_source"].fillna(0, inplace=True)

In [66]:
data_train["rating_value"].fillna(0, inplace=True)
data_test["rating_value"].fillna(0, inplace=True)

In [75]:
data_train["rating_count"].fillna(0, inplace=True)
data_test["rating_count"].fillna(0, inplace=True)

In [149]:
new_column = list()
for order in range(data_train.shape[0]):
    day = data_train["create_time_ts"][order].date().weekday()
    new_column.append(day)
data_train['day_of_week'] = new_column
new_column = list()
for order in range(data_test.shape[0]):
    day = data_test["create_time_ts"][order].date().weekday()
    new_column.append(day)
data_test['day_of_week'] = new_column

In [654]:
new_column = list()
for order in range(data_train.shape[0]):
    day = data_train["create_time_ts"][order].date().month
    new_column.append(day)
data_train['month'] = new_column
new_column = list()
for order in range(data_test.shape[0]):
    day = data_test["create_time_ts"][order].date().month
    new_column.append(day)
data_test['month'] = new_column

In [164]:
data_train["life_time"] = (data_train.create_time_ts - data_train.model_create_time_ts).astype('timedelta64[D]')
data_test["life_time"] = (data_test.create_time_ts - data_test.model_create_time_ts).astype('timedelta64[D]')

In [203]:
new_column = list()
for num in range(data_train.shape[0]):
    sess_key = data_train["sessionkey_id"][num]
    max_time = data_webstat[data_webstat["sessionkey_id"] == sess_key]["pageview_duration_sec"].max()
    new_column.append(max_time)
data_train['max_time_in_page'] = new_column
new_column = list()
for num in range(data_test.shape[0]):
    sess_key = data_test["sessionkey_id"][num]
    max_time = data_webstat[data_webstat["sessionkey_id"] == sess_key]["pageview_duration_sec"].max()
    new_column.append(max_time)
data_test['max_time_in_page'] = new_column

In [ ]:
data_train["max_time_in_page"].fillna(-1, inplace=True)
data_test["max_time_in_page"].fillna(-1, inplace=True)

In [263]:
new_column = list()
for num in range(data_train.shape[0]):
    sess_key = data_train["sessionkey_id"][num]
    count_page = data_webstat[data_webstat["sessionkey_id"] == sess_key].shape[0]
    new_column.append(count_page)
data_train['count_page'] = new_column
new_column = list()
for num in range(data_test.shape[0]):
    sess_key = data_test["sessionkey_id"][num]
    count_page = data_webstat[data_webstat["sessionkey_id"] == sess_key].shape[0]
    new_column.append(count_page)
data_test['count_page'] = new_column

In [264]:
data_train["count_page"].fillna(0, inplace=True)
data_test["count_page"].fillna(0, inplace=True)

In [229]:
new_column = list()
for num in range(data_train.shape[0]):
    sess_key = data_train["sessionkey_id"][num]
    count_page = data_webstat[data_webstat["sessionkey_id"] == sess_key]["pageview_duration_sec"].sum()
    new_column.append(count_page)
data_train['sum_time_page'] = new_column
new_column = list()
for num in range(data_test.shape[0]):
    sess_key = data_test["sessionkey_id"][num]
    count_page = data_webstat[data_webstat["sessionkey_id"] == sess_key]["pageview_duration_sec"].sum()
    new_column.append(count_page)
data_test['sum_time_page'] = new_column

In [230]:
data_train["sum_time_page"].fillna(0, inplace=True)
data_test["sum_time_page"].fillna(0, inplace=True)

In [244]:
new_column = list()
for num in range(data_train.shape[0]):
    sess_key = data_train["sessionkey_id"][num]
    count_page = data_webstat[data_webstat["sessionkey_id"] == sess_key]["pageview_duration_sec"].mean()
    new_column.append(count_page)
data_train['mean_time_page'] = new_column
new_column = list()
for num in range(data_test.shape[0]):
    sess_key = data_test["sessionkey_id"][num]
    count_page = data_webstat[data_webstat["sessionkey_id"] == sess_key]["pageview_duration_sec"].mean()
    new_column.append(count_page)
data_test['mean_time_page'] = new_column

In [245]:
data_train["mean_time_page"].fillna(0, inplace=True)
data_test["mean_time_page"].fillna(0, inplace=True)

In [333]:
data_webstat["page_type"].unique()

array([ 2,  5,  1,  7,  3,  4,  6,  9,  8, 10, 12, 13, 11])

In [319]:
DATA = data_webstat[data_webstat["page_type"] == 1]
new_column = list()
for num in range(data_train.shape[0]):
    sess_key = data_train["sessionkey_id"][num]
    count_page = DATA[DATA["sessionkey_id"] == sess_key].shape[0]
    new_column.append(count_page)
data_train['count_webpage_1'] = new_column
new_column = list()
for num in range(data_test.shape[0]):
    sess_key = data_test["sessionkey_id"][num]
    count_page = DATA[DATA["sessionkey_id"] == sess_key].shape[0]
    new_column.append(count_page)
data_test['count_webpage_1'] = new_column

In [320]:
data_train["count_webpage_1"].fillna(0, inplace=True)
data_test["count_webpage_1"].fillna(0, inplace=True)

In [334]:
DATA = data_webstat[data_webstat["page_type"] == 2]
new_column = list()
for num in range(data_train.shape[0]):
    sess_key = data_train["sessionkey_id"][num]
    count_page = DATA[DATA["sessionkey_id"] == sess_key].shape[0]
    new_column.append(count_page)
data_train['count_webpage_2'] = new_column
new_column = list()
for num in range(data_test.shape[0]):
    sess_key = data_test["sessionkey_id"][num]
    count_page = DATA[DATA["sessionkey_id"] == sess_key].shape[0]
    new_column.append(count_page)
data_test['count_webpage_2'] = new_column

In [335]:
data_train["count_webpage_2"].fillna(0, inplace=True)
data_test["count_webpage_2"].fillna(0, inplace=True)

In [344]:
DATA = data_webstat[data_webstat["page_type"] == 3]
new_column = list()
for num in range(data_train.shape[0]):
    sess_key = data_train["sessionkey_id"][num]
    count_page = DATA[DATA["sessionkey_id"] == sess_key].shape[0]
    new_column.append(count_page)
data_train['count_webpage_3'] = new_column
new_column = list()
for num in range(data_test.shape[0]):
    sess_key = data_test["sessionkey_id"][num]
    count_page = DATA[DATA["sessionkey_id"] == sess_key].shape[0]
    new_column.append(count_page)
data_test['count_webpage_3'] = new_column

In [345]:
data_train["count_webpage_3"].fillna(0, inplace=True)
data_test["count_webpage_3"].fillna(0, inplace=True)

In [358]:
DATA = data_webstat[data_webstat["page_type"] == 4]
new_column = list()
for num in range(data_train.shape[0]):
    sess_key = data_train["sessionkey_id"][num]
    count_page = DATA[DATA["sessionkey_id"] == sess_key].shape[0]
    new_column.append(count_page)
data_train['count_webpage_4'] = new_column
new_column = list()
for num in range(data_test.shape[0]):
    sess_key = data_test["sessionkey_id"][num]
    count_page = DATA[DATA["sessionkey_id"] == sess_key].shape[0]
    new_column.append(count_page)
data_test['count_webpage_4'] = new_column

In [359]:
data_train["count_webpage_4"].fillna(0, inplace=True)
data_test["count_webpage_4"].fillna(0, inplace=True)

In [368]:
DATA = data_webstat[data_webstat["page_type"] == 5]
new_column = list()
for num in range(data_train.shape[0]):
    sess_key = data_train["sessionkey_id"][num]
    count_page = DATA[DATA["sessionkey_id"] == sess_key].shape[0]
    new_column.append(count_page)
data_train['count_webpage_5'] = new_column
new_column = list()
for num in range(data_test.shape[0]):
    sess_key = data_test["sessionkey_id"][num]
    count_page = DATA[DATA["sessionkey_id"] == sess_key].shape[0]
    new_column.append(count_page)
data_test['count_webpage_5'] = new_column

In [369]:
data_train["count_webpage_5"].fillna(0, inplace=True)
data_test["count_webpage_5"].fillna(0, inplace=True)

In [378]:
DATA = data_webstat[data_webstat["page_type"] == 13]
new_column = list()
for num in range(data_train.shape[0]):
    sess_key = data_train["sessionkey_id"][num]
    count_page = DATA[DATA["sessionkey_id"] == sess_key].shape[0]
    new_column.append(count_page)
data_train['count_webpage_13'] = new_column
new_column = list()
for num in range(data_test.shape[0]):
    sess_key = data_test["sessionkey_id"][num]
    count_page = DATA[DATA["sessionkey_id"] == sess_key].shape[0]
    new_column.append(count_page)
data_test['count_webpage_13'] = new_column

In [379]:
data_train["count_webpage_13"].fillna(0, inplace=True)
data_test["count_webpage_13"].fillna(0, inplace=True)

In [407]:
DATA = data_webstat[data_webstat["page_type"] == 6]
new_column = list()
for num in range(data_train.shape[0]):
    sess_key = data_train["sessionkey_id"][num]
    count_page = DATA[DATA["sessionkey_id"] == sess_key].shape[0]
    new_column.append(count_page)
data_train['count_webpage_6'] = new_column
new_column = list()
for num in range(data_test.shape[0]):
    sess_key = data_test["sessionkey_id"][num]
    count_page = DATA[DATA["sessionkey_id"] == sess_key].shape[0]
    new_column.append(count_page)
data_test['count_webpage_6'] = new_column

In [408]:
data_train["count_webpage_6"].fillna(0, inplace=True)
data_test["count_webpage_6"].fillna(0, inplace=True)

In [431]:
DATA = data_webstat[data_webstat["page_type"] == 7]
new_column = list()
for num in range(data_train.shape[0]):
    sess_key = data_train["sessionkey_id"][num]
    count_page = DATA[DATA["sessionkey_id"] == sess_key].shape[0]
    new_column.append(count_page)
data_train['count_webpage_7'] = new_column
new_column = list()
for num in range(data_test.shape[0]):
    sess_key = data_test["sessionkey_id"][num]
    count_page = DATA[DATA["sessionkey_id"] == sess_key].shape[0]
    new_column.append(count_page)
data_test['count_webpage_7'] = new_column
data_train["count_webpage_7"].fillna(0, inplace=True)
data_test["count_webpage_7"].fillna(0, inplace=True)

In [455]:
DATA = data_webstat[data_webstat["page_type"] == 8]
new_column = list()
for num in range(data_train.shape[0]):
    sess_key = data_train["sessionkey_id"][num]
    count_page = DATA[DATA["sessionkey_id"] == sess_key].shape[0]
    new_column.append(count_page)
data_train['count_webpage_8'] = new_column
new_column = list()
for num in range(data_test.shape[0]):
    sess_key = data_test["sessionkey_id"][num]
    count_page = DATA[DATA["sessionkey_id"] == sess_key].shape[0]
    new_column.append(count_page)
data_test['count_webpage_8'] = new_column
data_train["count_webpage_8"].fillna(0, inplace=True)
data_test["count_webpage_8"].fillna(0, inplace=True)

In [471]:
DATA = data_webstat[data_webstat["page_type"] == 9]
new_column = list()
for num in range(data_train.shape[0]):
    sess_key = data_train["sessionkey_id"][num]
    count_page = DATA[DATA["sessionkey_id"] == sess_key].shape[0]
    new_column.append(count_page)
data_train['count_webpage_9'] = new_column
new_column = list()
for num in range(data_test.shape[0]):
    sess_key = data_test["sessionkey_id"][num]
    count_page = DATA[DATA["sessionkey_id"] == sess_key].shape[0]
    new_column.append(count_page)
data_test['count_webpage_9'] = new_column
data_train["count_webpage_9"].fillna(0, inplace=True)
data_test["count_webpage_9"].fillna(0, inplace=True)

In [496]:
DATA = data_webstat[data_webstat["page_type"] == 10]
new_column = list()
for num in range(data_train.shape[0]):
    sess_key = data_train["sessionkey_id"][num]
    count_page = DATA[DATA["sessionkey_id"] == sess_key].shape[0]
    new_column.append(count_page)
data_train['count_webpage_10'] = new_column
new_column = list()
for num in range(data_test.shape[0]):
    sess_key = data_test["sessionkey_id"][num]
    count_page = DATA[DATA["sessionkey_id"] == sess_key].shape[0]
    new_column.append(count_page)
data_test['count_webpage_10'] = new_column
data_train["count_webpage_10"].fillna(0, inplace=True)
data_test["count_webpage_10"].fillna(0, inplace=True)

In [505]:
DATA = data_webstat[data_webstat["page_type"] == 11]
new_column = list()
for num in range(data_train.shape[0]):
    sess_key = data_train["sessionkey_id"][num]
    count_page = DATA[DATA["sessionkey_id"] == sess_key].shape[0]
    new_column.append(count_page)
data_train['count_webpage_11'] = new_column
new_column = list()
for num in range(data_test.shape[0]):
    sess_key = data_test["sessionkey_id"][num]
    count_page = DATA[DATA["sessionkey_id"] == sess_key].shape[0]
    new_column.append(count_page)
data_test['count_webpage_11'] = new_column
data_train["count_webpage_11"].fillna(0, inplace=True)
data_test["count_webpage_11"].fillna(0, inplace=True)

In [521]:
DATA = data_webstat[data_webstat["page_type"] == 12]
new_column = list()
for num in range(data_train.shape[0]):
    sess_key = data_train["sessionkey_id"][num]
    count_page = DATA[DATA["sessionkey_id"] == sess_key].shape[0]
    new_column.append(count_page)
data_train['count_webpage_12'] = new_column
new_column = list()
for num in range(data_test.shape[0]):
    sess_key = data_test["sessionkey_id"][num]
    count_page = DATA[DATA["sessionkey_id"] == sess_key].shape[0]
    new_column.append(count_page)
data_test['count_webpage_12'] = new_column
data_train["count_webpage_12"].fillna(0, inplace=True)
data_test["count_webpage_12"].fillna(0, inplace=True)

In [578]:
# new_column = list()
# for num in range(data_train.shape[0]):
#     sess_key = data_train["sessionkey_id"][num]
#     count_page = data_webstat[data_webstat["sessionkey_id"] == sess_key].sort_values(by = "pageview_number", ascending = False)["page_type"].iloc[0]
#     new_column.append(count_page)
# data_train['last_page'] = new_column
# new_column = list()
# for num in range(data_test.shape[0]):
#     sess_key = data_test["sessionkey_id"][num]
#     count_page = data_webstat[data_webstat["sessionkey_id"] == sess_key].sort_values(by = "pageview_number", ascending = False)["page_type"].iloc[0]
#     new_column.append(count_page)
# data_test['last_page'] = new_column
# data_train["last_page"].fillna(0, inplace=True)
# data_test["last_page"].fillna(0, inplace=True)

In [681]:
DATA = data_webstat[data_webstat["page_type"] == 1]
new_column = list()
for num in range(data_train.shape[0]):
    sess_key = data_train["sessionkey_id"][num]
    count_page = DATA[DATA["sessionkey_id"] == sess_key]["pageview_duration_sec"].sum()
    new_column.append(count_page)
data_train['sum_min_1'] = new_column
new_column = list()
for num in range(data_test.shape[0]):
    sess_key = data_test["sessionkey_id"][num]
    count_page = DATA[DATA["sessionkey_id"] == sess_key]["pageview_duration_sec"].sum()
    new_column.append(count_page)
data_test['sum_min_1'] = new_column
data_train["sum_min_1"].fillna(0, inplace=True)
data_test["sum_min_1"].fillna(0, inplace=True)

In [690]:
DATA = data_webstat[data_webstat["page_type"] == 2]
new_column = list()
for num in range(data_train.shape[0]):
    sess_key = data_train["sessionkey_id"][num]
    count_page = DATA[DATA["sessionkey_id"] == sess_key]["pageview_duration_sec"].sum()
    new_column.append(count_page)
data_train['sum_min_2'] = new_column
new_column = list()
for num in range(data_test.shape[0]):
    sess_key = data_test["sessionkey_id"][num]
    count_page = DATA[DATA["sessionkey_id"] == sess_key]["pageview_duration_sec"].sum()
    new_column.append(count_page)
data_test['sum_min_2'] = new_column
data_train["sum_min_2"].fillna(0, inplace=True)
data_test["sum_min_2"].fillna(0, inplace=True)

In [700]:
DATA = data_webstat[data_webstat["page_type"] == 3]
new_column = list()
for num in range(data_train.shape[0]):
    sess_key = data_train["sessionkey_id"][num]
    count_page = DATA[DATA["sessionkey_id"] == sess_key]["pageview_duration_sec"].sum()
    new_column.append(count_page)
data_train['sum_min_3'] = new_column
new_column = list()
for num in range(data_test.shape[0]):
    sess_key = data_test["sessionkey_id"][num]
    count_page = DATA[DATA["sessionkey_id"] == sess_key]["pageview_duration_sec"].sum()
    new_column.append(count_page)
data_test['sum_min_3'] = new_column
data_train["sum_min_3"].fillna(0, inplace=True)
data_test["sum_min_3"].fillna(0, inplace=True)

In [724]:
DATA = data_webstat[data_webstat["page_type"] == 4]
new_column = list()
for num in range(data_train.shape[0]):
    sess_key = data_train["sessionkey_id"][num]
    count_page = DATA[DATA["sessionkey_id"] == sess_key]["pageview_duration_sec"].sum()
    new_column.append(count_page)
data_train['sum_min_4'] = new_column
new_column = list()
for num in range(data_test.shape[0]):
    sess_key = data_test["sessionkey_id"][num]
    count_page = DATA[DATA["sessionkey_id"] == sess_key]["pageview_duration_sec"].sum()
    new_column.append(count_page)
data_test['sum_min_4'] = new_column
data_train["sum_min_4"].fillna(0, inplace=True)
data_test["sum_min_4"].fillna(0, inplace=True)

In [733]:
DATA = data_webstat[data_webstat["page_type"] == 5]
new_column = list()
for num in range(data_train.shape[0]):
    sess_key = data_train["sessionkey_id"][num]
    count_page = DATA[DATA["sessionkey_id"] == sess_key]["pageview_duration_sec"].sum()
    new_column.append(count_page)
data_train['sum_min_5'] = new_column
new_column = list()
for num in range(data_test.shape[0]):
    sess_key = data_test["sessionkey_id"][num]
    count_page = DATA[DATA["sessionkey_id"] == sess_key]["pageview_duration_sec"].sum()
    new_column.append(count_page)
data_test['sum_min_5'] = new_column
data_train["sum_min_5"].fillna(0, inplace=True)
data_test["sum_min_5"].fillna(0, inplace=True)

In [742]:
DATA = data_webstat[data_webstat["page_type"] == 6]
new_column = list()
for num in range(data_train.shape[0]):
    sess_key = data_train["sessionkey_id"][num]
    count_page = DATA[DATA["sessionkey_id"] == sess_key]["pageview_duration_sec"].sum()
    new_column.append(count_page)
data_train['sum_min_6'] = new_column
new_column = list()
for num in range(data_test.shape[0]):
    sess_key = data_test["sessionkey_id"][num]
    count_page = DATA[DATA["sessionkey_id"] == sess_key]["pageview_duration_sec"].sum()
    new_column.append(count_page)
data_test['sum_min_6'] = new_column
data_train["sum_min_6"].fillna(0, inplace=True)
data_test["sum_min_6"].fillna(0, inplace=True)

DATA = data_webstat[data_webstat["page_type"] == 7]
new_column = list()
for num in range(data_train.shape[0]):
    sess_key = data_train["sessionkey_id"][num]
    count_page = DATA[DATA["sessionkey_id"] == sess_key]["pageview_duration_sec"].sum()
    new_column.append(count_page)
data_train['sum_min_7'] = new_column
new_column = list()
for num in range(data_test.shape[0]):
    sess_key = data_test["sessionkey_id"][num]
    count_page = DATA[DATA["sessionkey_id"] == sess_key]["pageview_duration_sec"].sum()
    new_column.append(count_page)
data_test['sum_min_7'] = new_column
data_train["sum_min_7"].fillna(0, inplace=True)
data_test["sum_min_7"].fillna(0, inplace=True)

In [908]:
data_webstat["time"] = None

In [920]:
data_webstat.time = data_webstat.apply(lambda x: datetime.strptime(x["date_time"][:19], '%Y-%m-%d %H:%M:%S'), axis=1)

In [921]:
data_webstat

,sessionkey_id,date_time,page_type,pageview_number,pageview_duration_sec,category_id,model_id,good_id,price,product_in_sale,time
0,122243978,1975-12-18 14:52:38.130,2,28,53.0,1773.0,NaN,NaN,NaN,NaN,1975-12-18 14:52:38
1,122243978,1975-12-18 15:02:32.403,5,36,163.0,NaN,NaN,NaN,NaN,NaN,1975-12-18 15:02:32
2,122243978,1975-12-18 15:08:11.147,5,40,156.0,NaN,NaN,NaN,NaN,NaN,1975-12-18 15:08:11
3,122243992,1975-12-18 14:23:33.177,1,4,565.0,3973.0,26706175.0,45171119.0,8436.0,1.0,1975-12-18 14:23:33
4,122243992,1975-12-18 14:32:58.763,1,5,73.0,3973.0,17297250.0,65195750.0,6624.0,1.0,1975-12-18 14:32:58
...,...,...,...,...,...,...,...,...,...,...,...
3334238,119215415,1975-12-03 14:45:42.320,1,2,NaN,141.0,856955.0,26681969.0,3467.0,1.0,1975-12-03 14:45:42
3334239,119215471,1975-12-03 14:45:30.520,6,5,21.0,NaN,NaN,NaN,NaN,NaN,1975-12-03 14:45:30
3334240,119215471,1975-12-03 14:50:11.470,6,8,16.0,NaN,NaN,NaN,NaN,NaN,1975-12-03 14:50:11
3334241,119215471,1975-12-03 14:50:56.133,6,10,7.0,NaN,NaN,NaN,NaN,NaN,1975-12-03 14:50:56


In [948]:
(data_train[data_train["sessionkey_id"] == 122243978]["create_time_ts"].max() - data_webstat[data_webstat["sessionkey_id"] == 122243978]['time'].max()).seconds

16

In [934]:
data_webstat[data_webstat["sessionkey_id"] == 122243978]['time'].max()

Timestamp('1975-12-18 15:10:47')

In [952]:
new_column = list()
for num in range(data_train.shape[0]):
    sess_key = data_train["sessionkey_id"][num]
    count_second = (data_train[data_train["sessionkey_id"] == sess_key]["create_time_ts"].max() - data_webstat[data_webstat["sessionkey_id"] == sess_key]['time'].max()).seconds
    new_column.append(count_second)
data_train['second_order_last'] = new_column
new_column = list()
for num in range(data_test.shape[0]):
    sess_key = data_test["sessionkey_id"][num]
    count_second = (data_test[data_test["sessionkey_id"] == sess_key]["create_time_ts"].max() - data_webstat[data_webstat["sessionkey_id"] == sess_key]['time'].max()).seconds
    new_column.append(count_second)
data_test['second_order_last'] = new_column

In [953]:
data_train

,order_id,good_id,price,utm_medium,utm_source,sessionkey_id,category_id,parent_id,root_id,model_id,...,count_webpage_12,month,sum_min_1,sum_min_2,sum_min_3,sum_min_4,sum_min_5,sum_min_6,sum_min_7,second_order_last
0,1269921,9896348,753,5,8.0,123777004,139,133,124,123517,...,0,12,0.0,0.0,0.0,0.0,0.0,0.0,0.0,545.0
1,1270034,9896348,753,1,2.0,123781654,139,133,124,123517,...,0,12,1439.0,1181.0,1163.0,0.0,15.0,0.0,0.0,86272.0
2,1268272,9896348,753,2,3.0,123591002,139,133,124,123517,...,0,12,256.0,21.0,6.0,0.0,15.0,0.0,0.0,76.0
3,1270544,9896348,753,1,1.0,123832302,139,133,124,123517,...,0,12,0.0,1926.0,0.0,0.0,0.0,0.0,0.0,85569.0
4,1270970,9896348,753,3,56.0,123881603,139,133,124,123517,...,0,12,10.0,19.0,18.0,0.0,0.0,47.0,0.0,233.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
104590,1250981,68657110,1497,4,1.0,121795335,2873,1229,1183,35231362,...,0,12,1425.0,0.0,70.0,1614.0,0.0,0.0,0.0,83726.0
104591,1173775,60391507,1354,1,2.0,114277105,2873,1229,1183,35231362,...,0,11,247.0,288.0,41.0,0.0,0.0,0.0,0.0,41690.0
104592,1180920,66971400,1452,2,0.0,114972131,2873,1229,1183,35231362,...,0,11,1197.0,261.0,105.0,0.0,0.0,39.0,0.0,86349.0
104593,1177882,60391498,1433,4,2.0,114672813,2873,1229,1183,35231362,...,0,11,1143.0,199.0,0.0,0.0,19.0,0.0,0.0,305.0


In [954]:
features = ["pics_qty", 
            "goods_qty", 
            "description_length", 
            "is_moderated", 
            "model_id", 
            "parent_id", 
            "category_id", 
            "utm_medium", 
            "price", 
            "root_id",
            "hour",
            "utm_source",
            "rating_value",
            "day_of_week",
            "life_time",
            "max_time_in_page",
            "count_page",
            "sum_time_page",
            "mean_time_page",
            "count_webpage_1",
            "count_webpage_2",
            "count_webpage_3",
            "count_webpage_4",
            "count_webpage_5",
            "count_webpage_6",
            "count_webpage_7",
            "month",
            "sum_min_1",
            "sum_min_3",
            "second_order_last"
           ]

# ОБУЧЕНИЕ (TEST)

In [955]:
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split
from catboost import cv
from catboost import Pool

In [956]:
X_train_val, X_test, y_train_val, y_test = train_test_split(data_train[features], data_train["is_callcenter"], test_size=0.15, random_state=42, stratify=data_train["is_callcenter"])
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.15, random_state=42, stratify = y_train_val)

In [957]:
model = CatBoostClassifier(eval_metric = 'AUC', class_weights = [0.35, 0.65], verbose = 100, iterations=1000)

In [958]:
model.fit(X_train, y_train,eval_set=(X_val, y_val), use_best_model=True)

Learning rate set to 0.092227
0:	test: 0.9095125	best: 0.9095125 (0)	total: 35.3ms	remaining: 35.2s
100:	test: 0.9366715	best: 0.9366715 (100)	total: 1.22s	remaining: 10.9s
200:	test: 0.9393171	best: 0.9393171 (200)	total: 2.28s	remaining: 9.07s
300:	test: 0.9405042	best: 0.9405042 (300)	total: 3.39s	remaining: 7.88s
400:	test: 0.9412858	best: 0.9412937 (398)	total: 4.44s	remaining: 6.64s
500:	test: 0.9417596	best: 0.9417882 (497)	total: 5.55s	remaining: 5.53s
600:	test: 0.9421279	best: 0.9421279 (600)	total: 6.62s	remaining: 4.39s
700:	test: 0.9423825	best: 0.9424457 (689)	total: 7.72s	remaining: 3.29s
800:	test: 0.9424033	best: 0.9425450 (779)	total: 8.88s	remaining: 2.21s
900:	test: 0.9424631	best: 0.9425450 (779)	total: 10.2s	remaining: 1.12s
999:	test: 0.9425805	best: 0.9426108 (987)	total: 11.3s	remaining: 0us

bestTest = 0.9426108389
bestIteration = 987

Shrink model to first 988 iterations.


In [855]:
model.set_probability_threshold(0.39)

In [959]:
predictions = model.predict(X_test[features])

In [960]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test, predictions)

0.8571215600146418

# ОБУЧЕНИЕ (Kaggle)

In [961]:
X_train, X_val, y_train, y_val = train_test_split(data_train[features], data_train["is_callcenter"], test_size=0.2, random_state=42, stratify=data_train["is_callcenter"])

In [962]:
model = CatBoostClassifier(eval_metric = 'AUC', class_weights = [0.35, 0.65], verbose = 100)
model.fit(X_train, y_train,eval_set=(X_val, y_val), use_best_model=True)

Learning rate set to 0.094578
0:	test: 0.9086783	best: 0.9086783 (0)	total: 29.7ms	remaining: 29.6s
100:	test: 0.9368511	best: 0.9368511 (100)	total: 1.27s	remaining: 11.3s
200:	test: 0.9400637	best: 0.9400637 (200)	total: 2.42s	remaining: 9.63s
300:	test: 0.9416127	best: 0.9416302 (298)	total: 3.69s	remaining: 8.57s
400:	test: 0.9424825	best: 0.9424825 (400)	total: 4.87s	remaining: 7.27s
500:	test: 0.9431896	best: 0.9431929 (499)	total: 6.1s	remaining: 6.07s
600:	test: 0.9436684	best: 0.9436684 (600)	total: 7.25s	remaining: 4.81s
700:	test: 0.9441349	best: 0.9441481 (695)	total: 8.41s	remaining: 3.59s
800:	test: 0.9445612	best: 0.9445907 (789)	total: 9.57s	remaining: 2.38s
900:	test: 0.9447286	best: 0.9447569 (892)	total: 10.7s	remaining: 1.18s
999:	test: 0.9449687	best: 0.9449780 (998)	total: 12.1s	remaining: 0us

bestTest = 0.9449780197
bestIteration = 998

Shrink model to first 999 iterations.


In [963]:
predictions = model.predict(data_test[features])

In [964]:
id_column = data_test["order_id"]
result_column = predictions
dataset = pd.DataFrame({'order_id': id_column, 'is_callcenter': list(result_column)}, columns=['order_id', 'is_callcenter'])
dataset.to_csv("sample_submission.csv", index=False)